<a href="https://colab.research.google.com/github/suhbaca/AI4GOOD_TeamProject/blob/suh/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **LLAMA 2 Model**





In [ ]:
#access huggingface
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
#implement a pipeline that will allow us to give prompts and get responses

import torch
import transformers

pipeline = transformers.pipeline(
"text-generation",

model=model,

tokenizer=tokenizer,

torch_dtype=torch.float16,

device_map="auto",

)

In [ ]:
# create a sequence that will take in a prompt

#changing the num_return_sequence will give 3 different possible outputs for the same prompt

response_template= " Answer the prompt in the following template : word1, word2, word3, word4, word5"
x= 'The user is trying to find a word that matches the description in the prompt. The answer is 5 possible words that the prompt is trying to describe. The following are examples of Prompt-Answer format.\n'

few_shot = "Prompt: it’s a type of bird, kind of rare. They’re very colorful, with long tails, and they can mimic sounds. They live in tropical areas\n Answer:  Hummingbird, Parrot, Finch, Toucan, Macaw\n Prompt: It’s a type of bird, very colorful, and it can mimic human speech. People sometimes keep them as pets. They live in tropical areas.\nAnswer: Parrot, Macaw, Cockatoo, Canary, Finch\nPrompt: I need the...you know...the thing for writing\nAnswer: Pen, Pencil, Quill, Fountain Pen, Crayon\nPrompt : I want to go to the...um...place where we get groceries\n  Answer: Store, Supermarket, Market, Grocery, Shop\nPrompt : "

y = 'Now it is your turn to give the answer to the last prompt. Give 30 possible words being described instead of just 5 like in the examples. It is important that you give 30 words. It is very important that you only provide the final output without any additional comments or remarks or notes.'
sequences = pipeline(
x + few_shot+ 'Where is my...uh...the thing you use to open doors\n' + y,

do_sample=True,

top_k=20,

num_return_sequences=1,

eos_token_id=tokenizer.eos_token_id,

max_length=800,

temperature= 0.3,

return_full_text= False

)
print(sequences)
for seq in sequences:
  print(1)
  print(f"{seq['generated_text']}")

In [ ]:
import ir_datasets
dataset = ir_datasets.load("trec-tot/2023/dev")
dataset.docs_iter()[:1]
#for query in dataset.queries_iter():
   # query # namedtuple<query_id, url, domain, title, text, sentence_annotations>

In [ ]:
prompt_template = """You are a helpful assistant. You do not respond as 'User' or pretend to be 'User'. You only
respond once as 'Assistant'
Question: {question}
Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
def main():
    # Instantiate the chain for that user session
    prompt = PromptTemplate(template=prompt_template, input_variables=["question"])
    llm_chain = LLMChain(prompt=prompt, llm=model, verbose=True)

    # Store the chain in the user session
    cl.user_session.set("llm_chain", llm_chain)



async def main(message: str):
    # Retrieve the chain from the user session
    llm_chain = cl.user_session.get("llm_chain")  # type: LLMChain

    # Call the chain asynchronously
    res = await llm_chain.acall(message, callbacks=[cl.AsyncLangchainCallbackHandler()])

    # Do any post processing here

    # "res" is a Dict. For this chain, we get the response by reading the "text" key.
    # This varies from chain to chain, you should check which key to read.
    await cl.Message(content=res["text"]).send()

In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.793s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!chainlit hello
!chainlit run app.py &>/content/logs.txt &

2024-06-11 16:28:30 - Your app is available at http://localhost:8000
